In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
import time
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from src import settings
import progressbar

retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'

In [19]:
# events_df = pd.read_csv(retail_rocket_root_path + 'raw_data/events.csv')
# category_tree_df = pd.read_csv(retail_rocket_root_path + 'raw_data/category_tree.csv')

# item_properties file 
# 417 053 unique items
# every row in the file has corresponding timestamp
# item properties file contain timestamp column because all of them are time dependent
# since properties may change over time, e.g. price, category, etc

# value of categoryid properties contain item category identifier
# value of available properties contains availability of the item ( 1: available, 0: ow)
# All numerical values were marked with "n" char at the beginning, and have 3 digits precision after decimal point


item_properties_1_df = pd.read_csv(retail_rocket_root_path + 'raw_data/item_properties_part1.csv')
item_properties_2_df = pd.read_csv(retail_rocket_root_path + 'raw_data/item_properties_part2.csv')
# timestamp,itemid,property,value

# 1435460400000,460429,categoryid,1338
# 1441508400000,206783,888,1116713 960601 n277.200
# 1439089200000,395014,400,n552.000 639502 n720.000 424566
# 1431226800000,59481,790,n15360.000
# 1431831600000,156781,917,828513
# 1436065200000,285026,available,0
# 1434250800000,89534,213,1121373
# 1431831600000,264312,6,319724
# 1433646000000,229370,202,1330310
# 1434250800000,98113,451,1141052 n48.000
# 1439089200000,450113,888,1038400 45956 n504.000

In [20]:
item_properties_1_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [21]:
item_properties_2_df.head()

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062


In [22]:
# loc function: access a group of row and columns by labels.

# item_properties_1_df.property: dua ra item_id va properties cua item_properties_1_df
# 0           categoryid
# 1                  888
# 2                  400
# 3                  790
# 4                  917

# item_properties_1_df.property == 'categoryid'
#  0            True
#  1           False
#  2           False
#  3           False
#  4           False
#  5           False
#  6           False

# item_properties_1_df.loc[item_properties_1_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]

# lấy ở item_properties_1_df voi property = categoryid voi cac gia tri tren cac cot timestamp, itemid va value

# giam gia tri tu 10999999 xuong con 426305 ban ghi

# Value of the "categoryid" property contains item category identifier.

item_cat_1 = item_properties_1_df.loc[item_properties_1_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]
item_cat_1.head()

,timestamp,itemid,value
0,1435460400000,460429,1338
140,1432436400000,281245,1277
151,1435460400000,35575,1059
189,1437274800000,8313,1147
197,1437879600000,55102,47


In [23]:
item_cat_2 = item_properties_2_df.loc[item_properties_2_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]
item = item_cat_1.append(item_cat_2)
# y = item.drop_duplicates(['itemid', 'value']).groupby(['itemid', 'value']).count()
# y.head()
# # temp = item.loc[item.itemid == 25]
# # temp.sort_values(by=['timestamp'])
# # y = item.groupby(['itemid', 'value']).count()
# y.loc[y.timestamp == 1]

In [24]:
print "Record retrive from 1:", len(item_cat_1)
print "Record retrive from 1:", len(item_cat_2)
print "Total records:", len(item)
print(item[:10])

Record retrive from 1: 426305
Record retrive from 1: 361909
Total records: 788214
         timestamp  itemid value
0    1435460400000  460429  1338
140  1432436400000  281245  1277
151  1435460400000   35575  1059
189  1437274800000    8313  1147
197  1437879600000   55102    47
213  1433041200000  397079   619
237  1436670000000  265036  1228
254  1437879600000  124459  1277
310  1437879600000  350508   546
325  1439089200000  221365  1226


In [25]:
# y.loc[:, ['itemid', 'value']] # type: pd.DataFrame

# xoa cac ban ghi trung nhau tren cac cot item_id va value, va loc lai lay cac gia tri tren 2 cot itemid va value.
z = item.drop_duplicates(['itemid', 'value']).loc[:, ['itemid', 'value']]
print z.head()
print "\n"
print "Length of items after remove duplicate:", len(z)
# y.to_dict()
# z.to_csv(retail_rocket_root_path + 'temp.txt', index=False, header=False)

     itemid value
0    460429  1338
140  281245  1277
151   35575  1059
189    8313  1147
197   55102    47


Length of items after remove duplicate: 442432


In [26]:
# convert i2index.txt into dictionary
with open(retail_rocket_root_path + 'i2index.txt') as f: 
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}

# get list key from this dict.
active_item_list = active_item_dict.keys()

In [27]:
# remove cac phan tu trong z khong nam trong active item list (cac key lay tu file i2index.txt)
# z: item properties combine from 2 file.
s = z[z.itemid.isin(active_item_list)]
print 's head'
print s.head()

y = s.loc[:, 'itemid'].drop_duplicates() # xoa item_id trung nhau va loc tren cot itemid
print "y data type:",type(y)
print (y.head())
print len(y)


s head
     itemid value
189    8313  1147
408   96660   498
504  244437   438
804  313481  1613
836  366108   498
y data type: <class 'pandas.core.series.Series'>
189      8313
408     96660
504    244437
804    313481
836    366108
Name: itemid, dtype: int64
83274


In [16]:
active_item_dict[8313]

75509

In [28]:
# apply len tat ca cac phan tu cua s voi ham vo danh lambda.
# s.head()
#      itemid value
# 189    8313  1147
# 408   96660   498
# 504  244437   438
# 804  313481  1613
# 836  366108   498

# active item dict
# 393218: 27821,
#  393219: 30596,
#  393220: 2768,
#  152918: 36242,
#  6: 5054,

# thay the cac phan tu tung hang trong s tuong ung boi active_item_dict[x[0]], x[1]
# t moi thu duoc bao gom key cua active_item_dict va value cua s tuong ung.
t = s.apply(lambda x: [active_item_dict[x[0]], x[1]], axis=1, result_type='expand')
t.head()
# s.to_csv(retail_rocket_root_path + 'i2pcat.txt', index=False, header=False)

,0,1
189,75509,1147
408,42912,498
504,13362,438
804,33257,1613
836,60227,498


In [30]:
t.to_csv(retail_rocket_root_path + 'i2pcat.txt', index=False, header=False)
# i2pcat: item_id va categories tuong ung cua item do.
# voi item_id la item_id moi cua item khi chua mapping

In [31]:
# y la cac old item id.
print y.head()
x = y.to_list() # lay cac old item_id trong y
print len(x)
# print x[:5]

189      8313
408     96660
504    244437
804    313481
836    366108
Name: itemid, dtype: int64
83274


# Reading from event.csv

In [32]:
events_df = pd.read_csv(retail_rocket_root_path + 'raw_data/events.csv')
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [33]:
# get even_df tu trong danh sach cac events_df ma itemid do nam trong x (x la old item id) 
# (loai cac sk co cac item ko co trong x)
new_event_df = events_df[events_df.itemid.isin(x)]
# print(new_event_df.count()) -> 2094466
# print(events_df.count()) -> 2756101
new_event_df.to_csv(retail_rocket_root_path + 'raw_data/new_events.csv', index=False)
new_event_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
4,1433221337106,951259,view,367447,NaN
6,1433221923240,810725,view,443030,NaN
7,1433223291897,794181,view,439202,NaN


In [34]:
category_tree_df = pd.read_csv(retail_rocket_root_path + 'raw_data/category_tree.csv')
category_tree_df.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [35]:
# print category_tree_df['categoryid']
print type(category_tree_df['categoryid'])

<class 'pandas.core.series.Series'>


In [36]:
# lay ra cac phan tu trong category_tree_df ma parent la na. (ASK NHAT)????
category_tree_df.loc[category_tree_df.parentid.isna()]

,categoryid,parentid
5,231,NaN
589,791,NaN
861,1490,NaN
939,431,NaN
1111,755,NaN
1142,378,NaN
1208,1579,NaN
1211,1394,NaN
1322,659,NaN
1329,1057,NaN


# Main: After that it change to retail_rocket data

In [45]:
retail_rocket_df = pd.read_csv(retail_rocket_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(retail_rocket_df.head())
# active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|', skiprows=1)
# active_item_df.head()
# ASK NHAT
# _explicit.clean.txt: uid la user_id ban dau
# itemid la item_id ban dau

      uid  itemid           time
0  287857    5206  1433223236124
1  158090   10572  1433221078505
2  599528  356475  1433221941632
3  599528  356475  1433222276276
4  105775  312728  1433220880956


In [46]:
# load list key for i2index
with open(retail_rocket_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print "Number of items: ",len(active_item_list)

Number of items:  83274


In [47]:
# list key index from u2index.
with open(retail_rocket_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print "Number of users: ",len(active_user_list)

Number of users:  36751


In [50]:
print "length before filter: ", len(retail_rocket_df)
# loai bo tuong tac ma user khong co trong active_user_list
retail_rocket_df = retail_rocket_df[retail_rocket_df.uid.isin(active_user_list)]
retail_rocket_df = retail_rocket_df[retail_rocket_df.itemid.isin(active_item_list)]
print "length after filter: ", len(retail_rocket_df)
retail_rocket_df.head()

length before filter:  39939
length after filter:  39939


,uid,itemid,time
6,121688,15335,1433193500981
9,168685,111530,1433218757551
14,1203684,80953,1433217911121
16,273406,214912,1433174825959
17,615793,355994,1433175546693


In [64]:
print 'Sorting by uid and time giam dan (cang xa so voi hien tai):'
# a = retail_rocket_df.sort_values(by=['uid'])
retail_rocket_df_sort = retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time"], ascending = False)).reset_index(drop=True)
print (retail_rocket_df_sort[:50])

Sorting by uid and time giam dan (cang xa so voi hien tai):
     uid  itemid           time
0    155  389974  1442032680026
1    155  442601  1441952265950
2    155   41882  1441952264613
3    155  181405  1441951967871
4    155  452082  1441951937583
5    172   10034  1439602141686
6    172  465522  1439602141230
7    172  465522  1439601228324
8    172  465522  1439601219691
9    172   10034  1439599816912
10   318  133814  1435607785773
11   627  414182  1440103716892
12   627  414182  1440099961936
13   645   37029  1436226279887
14   648  207125  1436066053539
15   795  207825  1439244127846
16   795  207825  1439243682003
17   878  283916  1437716162516
18  2004  132772  1432016623960
19  2019  445559  1433368510965
20  2019  393144  1433368509996
21  2019  393144  1433316439898
22  2019  445559  1433316361979
23  2019  328869  1433311895274
24  2114  342264  1431908831703
25  2242  402047  1434339796245
26  2242  402047  1434335993380
27  2326   87702  1430690749035
28  3104  34

In [65]:
active_user_dict[155]

16518

In [67]:
# thay the rating ban dau boi user_id, item_id moi trong u2index va i2index.
retail_rocket_df_sort = retail_rocket_df_sort.apply(lambda x: [active_user_dict[x[0]], active_item_dict[x[1]]], axis=1, result_type='expand')
print(retail_rocket_df_sort[:50])

        0      1
0   16518  24797
1   16518  67373
2   16518  58774
3   16518  22559
4   16518  69597
5   16975   1400
6   16975   3598
7   16975   3598
8   16975   3598
9   16975   1400
10  12108   3382
11  18350  47924
12  18350  47924
13  31840   5298
14  30912   7015
15  15769  16973
16  15769  16973
17  34669   5982
18  21261   9076
19   1778   1729
20   1778   1540
21   1778   1540
22   1778   1729
23   1778  16390
24  27508   2468
25   7297  21207
26   7297  21207
27  23732  59669
28  27500  66385
29  27500  66537
30  27500  66537
31  27500  66385
32   9679  25684
33   9679  29037
34   9679  29364
35   9679  29037
36   9679  29364
37   9679  29037
38   9679  29364
39   9679  29037
40  22063  40930
41  22063   7042
42  22375  56807
43   7001  63987
44   7001  63987
45  18768   2722
46  18768   2722
47  18768  49336
48  28355  26885
49  28355   7677


In [42]:
print active_user_dict[121688]
print active_item_dict[15335]

31
59933


# Sorting ratings.txt by time of implicit feedback

In [3]:
retail_rocket_df = pd.read_csv(retail_rocket_root_path + 'ratings.txt', header=None, names=['uid', 'itemid', 'time_implicit','number_im','time_explicit','number_ex'])
print(retail_rocket_df.head())

   uid  itemid  time_implicit  number_im  time_explicit  number_ex
0    0       0  1433224214164          1             -1          0
1    0    2921  1433224672007          1             -1          0
2    0    5692  1433225555976          1             -1          0
3    0    6353  1436503481324          4             -1          0
4    0    9442  1433730019241          2             -1          0


In [5]:
print 'Sorting by uid and time tang dan (tu qua khu cho den bay gio):'
# a = retail_rocket_df.sort_values(by=['uid'])
retail_rocket_df_sort = retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time_implicit"], ascending = True)).reset_index(drop=True)
print (retail_rocket_df_sort[:50])

Sorting by uid and time giam dan (cang xa so voi hien tai):
    uid  itemid  time_implicit  number_im  time_explicit  number_ex
0     0     919  1432781848232          1             -1          0
1     0    4244  1432782322169          1             -1          0
2     0       0  1433224214164          1             -1          0
3     0    2921  1433224672007          1             -1          0
4     0    5692  1433225555976          1             -1          0
5     0   10195  1433480149682          1             -1          0
6     0    1249  1433480524891          1             -1          0
7     0   10652  1433703062882          1             -1          0
8     0   10345  1433729380709          1             -1          0
9     0   10340  1433729489683          1             -1          0
10    0    9442  1433730019241          2             -1          0
11    0    8738  1433730231553          1             -1          0
12    0   13742  1433730402675          1             -1

In [6]:
# in ra file 
retail_rocket_df_sort.to_csv(retail_rocket_root_path + 'ratings_sort_tuc.txt', index=False, header=False)

# user item explicit using for trainning without time series

In [43]:
#retail_rocket_df.to_csv(retail_rocket_root_path + 'ui_ex.xt', index=False, header=False)

In [68]:
# user item explicit for trainning with timestamp
retail_rocket_df_sort.to_csv(retail_rocket_root_path + 'ui_ex_timestamp_v2.txt', index=False, header=False)

In [ ]:
# Generate using implicit dataset into ui_im_timestamp.txt

In [71]:
retail_rocket_df_implicit = pd.read_csv(retail_rocket_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(retail_rocket_df.head())

       uid  itemid           time
0  257597  355908  1433221332117
1  992329  248676  1433224214164
2  951259  367447  1433221337106
3  810725  443030  1433221923240
4  794181  439202  1433223291897


In [72]:
print "length before filter: ", len(retail_rocket_df_implicit)
# loai bo tuong tac ma user khong co trong active_user_list
retail_rocket_df_implicit = retail_rocket_df_implicit[retail_rocket_df_implicit.uid.isin(active_user_list)]
retail_rocket_df_implicit = retail_rocket_df_implicit[retail_rocket_df_implicit.itemid.isin(active_item_list)]
print "length after filter: ", len(retail_rocket_df_implicit)
retail_rocket_df_implicit.head()

length before filter:  2020318
length after filter:  573756


,uid,itemid,time
1,992329,248676,1433224214164
5,824915,428805,1433220899221
13,1398644,135256,1433224070841
16,125625,17655,1433223203944
18,1076270,262799,1433222147345


In [73]:
print 'Sorting by uid and time giam dan (cang xa so voi hien tai):'
# a = retail_rocket_df.sort_values(by=['uid'])
retail_rocket_df_implicit_sort = retail_rocket_df_implicit.groupby(["uid"]).apply(lambda x: x.sort_values(["time"], ascending = False)).reset_index(drop=True)
print (retail_rocket_df_implicit_sort[:5])

Sorting by uid and time giam dan (cang xa so voi hien tai):
   uid  itemid           time
0   51  279059  1441495684069
1   51   49967  1441495634886
2   51  198762  1441495582729
3   51  429304  1441495416363
4   51  198762  1441495318252


In [74]:
# thay the rating ban dau boi user_id, item_id moi trong u2index va i2index.
retail_rocket_df_implicit_sort = retail_rocket_df_implicit_sort.apply(lambda x: [active_user_dict[x[0]], active_item_dict[x[1]]], axis=1, result_type='expand')
print(retail_rocket_df_implicit_sort[:50])

        0      1
0   21616  42605
1   21616  13063
2   21616   7821
3   21616  13153
4   21616   7821
5   21616  54900
6   23498   3389
7   23498  50849
8   23498  47531
9   23498  47531
10  23498   8883
11  23498  59189
12  23498    244
13  23498   8883
14  23498   3389
15  23498    244
16  23498  59189
17   6738  21505
18   6738  21505
19   6738    895
20   6738  21434
21   6738  21638
22   6738  21195
23   6738  18111
24  16518  53110
25  16518   9040
26  16518   2877
27  16518  23223
28  16518  13380
29  16518   1754
30  16518  13380
31   2113  51263
32   2113   1107
33   2113   8117
34   2113   7287
35   2113   7288
36  16975   3598
37  16975   3598
38  16975   3598
39  16975  21354
40  16975  19235
41  16975  11370
42  16975  33805
43  16975   1400
44  16975  16840
45  16975  33103
46  16975  33103
47  16975  33103
48  16975   1400
49  16975  21015


In [75]:
# user item explicit for trainning with timestamp
retail_rocket_df_implicit_sort.to_csv(retail_rocket_root_path + 'ui_im_timestamp_v2.txt', index=False, header=False)

In [ ]:
# Running with time sequences.

In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
import time
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from src import settings
from sklearn import metrics

retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'

In [2]:
# khao sat tu bo data ban dau.
implicit_retail_rocket_df = pd.read_csv(retail_rocket_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(implicit_retail_rocket_df.head())

      uid  itemid           time
0  257597  355908  1433221332117
1  992329  248676  1433224214164
2  951259  367447  1433221337106
3  810725  443030  1433221923240
4  794181  439202  1433223291897


In [3]:
# load list key for i2index
with open(retail_rocket_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print "Number of items: ",len(active_item_list)

Number of items:  83274


In [4]:
# list key index from u2index.
with open(retail_rocket_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print "Number of users: ",len(active_user_list)

Number of users:  36751


In [5]:
print "length before filter: ", len(implicit_retail_rocket_df)
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
implicit_retail_rocket_df = implicit_retail_rocket_df[implicit_retail_rocket_df.uid.isin(active_user_list)]
implicit_retail_rocket_df = implicit_retail_rocket_df[implicit_retail_rocket_df.itemid.isin(active_item_list)]
print "length after filter: ", len(implicit_retail_rocket_df)

length before filter:  2020318
length after filter:  573756


In [6]:
implicit_retail_rocket_df = implicit_retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["uid","time"], ascending = True)).reset_index(drop=True)

In [7]:
implicit_retail_rocket_df[:20]

,uid,itemid,time
0,51,358388,1441495257511
1,51,198762,1441495318252
2,51,429304,1441495416363
3,51,198762,1441495582729
4,51,49967,1441495634886
5,51,279059,1441495684069
6,54,388096,1442338531705
7,54,283115,1442338665550
8,54,38965,1442338748248
9,54,319680,1442338841333


In [8]:
out_file = retail_rocket_root_path + "_implicit.filter.clean.txt"
implicit_retail_rocket_df.to_csv(out_file, header=None, index=None, sep=',')

# explicit dataset for preprocessing.

In [9]:
explicit_retail_rocket_df = pd.read_csv(retail_rocket_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(explicit_retail_rocket_df.head())

      uid  itemid           time
0  287857    5206  1433223236124
1  158090   10572  1433221078505
2  599528  356475  1433221941632
3  599528  356475  1433222276276
4  105775  312728  1433220880956


In [10]:
print "length before filter: ", len(explicit_retail_rocket_df)
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
explicit_retail_rocket_df = explicit_retail_rocket_df[explicit_retail_rocket_df.uid.isin(active_user_list)]
explicit_retail_rocket_df = explicit_retail_rocket_df[explicit_retail_rocket_df.itemid.isin(active_item_list)]
print "length after filter: ", len(explicit_retail_rocket_df)

length before filter:  88886
length after filter:  39939


In [11]:
explicit_retail_rocket_df = explicit_retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["uid","time"], ascending = True)).reset_index(drop=True)

In [12]:
explicit_retail_rocket_df[:20]

,uid,itemid,time
0,155,452082,1441951937583
1,155,181405,1441951967871
2,155,41882,1441952264613
3,155,442601,1441952265950
4,155,389974,1442032680026
5,172,10034,1439599816912
6,172,465522,1439601219691
7,172,465522,1439601228324
8,172,465522,1439602141230
9,172,10034,1439602141686


In [13]:
out_file = retail_rocket_root_path + "_explicit.filter.clean.txt"
explicit_retail_rocket_df.to_csv(out_file, header=None, index=None, sep=',')

In [20]:
# from _implicit.filter.clean.txt, _explicit.filter.clean.txt merge into one.
implicit_file = retail_rocket_root_path + "_implicit.filter.clean.txt"
explicit_file = retail_rocket_root_path + "_explicit.filter.clean.txt"

implicit_interacted_dict = {}
file_pointer = open(implicit_file, "r")  # reading from _implicit.clean.txt
    # b15f9e5, d9886b1, 1471066727
    # 8007201, e295b87, 1471156894

csv_reader = csv.reader(file_pointer, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
for line in csv_reader:
    raw_uid = line[0]  # string
    raw_item_id = line[1]
    if raw_uid in implicit_interacted_dict:
        implicit_interacted_dict[raw_uid].add(raw_item_id)
    else:
        implicit_interacted_dict[raw_uid] = {raw_item_id}

In [22]:
implicit_interacted_dict["54"]

{'249114', '283115', '319680', '388096', '38965', '442228'}

In [ ]:
# from _implicit.clean.txt, _explicit.clean.txt
# output: user_index_output, item_index_ouput
# ratings.txt
def gen_ratings_data_with_explicit(implicit_input, explicit_input, user_index_output, item_index_output, 
                                   output):
    # create uid blacklist
    interacted_dict = {}

    file_pointer = open(implicit_input, "r")  # reading from _implicit.clean.txt
    # b15f9e5, d9886b1, 1471066727
    # 8007201, e295b87, 1471156894

    csv_reader = csv.reader(file_pointer, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
    for line in csv_reader:
        raw_uid = line[0]  # string
        raw_item_id = line[1]
        if raw_uid in interacted_dict:
            interacted_dict[raw_uid].add(raw_item_id)  # tao ra bo tu dien interact cho moi user.
            # co cau truc kieu uid => list item interacted by uid
        else:
            # interacted_dict[raw_uid] = type set()
            interacted_dict[raw_uid] = {raw_item_id}  # neu user chua co trong interacted_dict thi tao ra dict moi

    uid_blacklist = set()  # tap cac user co so luong item implicit tuong tac nho hon 5
    for uid in interacted_dict:  # moi user trong interacted_list
        if len(interacted_dict[uid]) < 5:  # so luong nho hon 5 thi them user do vao  black list
            uid_blacklist.add(uid)
    file_pointer.close()
    print("len uid_blacklist:", len(uid_blacklist))

    # convert uid, item_id
    logging.info("convert uid, item_id")

    # uid_dict, item_id_dict la anh xa tu raw_id sang id dang so nguyen tu 0 den n
    user_id_dict = {}
    item_id_dict = {}

    implicit_rating_dict = {}
    uid_count = 0
    item_id_count = 0

    file_pointer = open(implicit_input, "r")  # read file implict_clean
    csv_reader = csv.reader(file_pointer, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
    # b15f9e5, d9886b1, 1471066727
    # 8007201, e295b87, 1471156894
    for line in csv_reader:
        raw_uid = line[0]  # string
        if raw_uid not in uid_blacklist:  # khong nam trong black list
            raw_item_id = line[1]  # item id
            raw_timestamp = line[2]  # time click
            if raw_uid not in user_id_dict:  # tao ra dict voi key la user_id => stt cua user do.
                user_id_dict[raw_uid] = uid_count
                uid_count += 1
            if raw_item_id not in item_id_dict:  # dict item: key la item_id => so tt cua item do
                item_id_dict[raw_item_id] = item_id_count
                item_id_count += 1

            # add user data to dict
            uid = user_id_dict[raw_uid]
            # int : lay ra uid moi cua user tuong ung voi raw_uid
            if uid in implicit_rating_dict:
                implicit_rating_dict[uid].append((item_id_dict[raw_item_id], raw_timestamp))
                # implicit_rating_dict: dict voi key la uid va value la mot mang voi moi phan tu la new_item_id, time_stamp
            else:
                implicit_rating_dict[uid] = [(item_id_dict[raw_item_id], raw_timestamp)]
    file_pointer.close()
    print("len user list:", len(user_id_dict))
    print("len item list:", len(item_id_dict))
    print("len implicit_rating_dict:", len(implicit_rating_dict))

    with open(user_index_output, 'w') as f:  # viet ra file 'u2index.txt'
        csv_writer = csv.writer(f, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
        for k in user_id_dict:
            csv_writer.writerow([k, user_id_dict[k]])  # uid ban dau => uid moi.

    with open(item_index_output, 'w') as f:  # viet ra file 'i2index.txt'
        csv_writer = csv.writer(f, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
        for k in item_id_dict:
            csv_writer.writerow([k, item_id_dict[k]])  # item id raw => itemid moi.
    # load explicit data put to explicit_rating_dict

    explicit_rating_dict = {}
    file_pointer = open(explicit_input, "r")  # reading from '_explicit.clean.txt'

    csv_reader = csv.reader(file_pointer, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
    for line in csv_reader:  # moi dong trong bo du lieu explicit.
        raw_uid = line[0]  # raw uid
        raw_item_id = line[1]  # raw item id
        raw_timestamp = line[2]  # raw time stamp
        if (raw_uid in user_id_dict) and (raw_item_id in item_id_dict):
            uid = user_id_dict[raw_uid]  # id moi cua user
            if uid in explicit_rating_dict:  # tao ra explicit rating dict cho moi user.
                # voi key la uid moi (xay tu bo du lieu implicit
                # value tuong ung la item_id moi va raw time stamp
                explicit_rating_dict[uid].append((item_id_dict[raw_item_id], raw_timestamp))
            else:
                explicit_rating_dict[uid] = [(item_id_dict[raw_item_id], raw_timestamp)]
    file_pointer.close()

    # gen ratings data
    logging.info('gen ratings with explicit data')
    out_file = open(output, 'w')  # writing to file ratings.txt
    csv_writer = csv.writer(out_file, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)  # ratings.txt
    # format outfile rating.txt: raw_uid,raw_item_id,implicit_timestamp,num_implicicit,explicit_timestamp,num_explicit
    for uid in implicit_rating_dict:  # moi uid trong implicit rating dict.
        # check duplicate
        implicit_item_time_dict = {}
        explicit_item_time_dict = {}
        for his in implicit_rating_dict[uid]:  # implicit_rating_dict[uid]: mang cac cap new_itemid, time stamp
            item_id = his[0]
            timestamp = his[1]
            if item_id in implicit_item_time_dict:  # neu item_id nam trong implicit_item_time_dict
                if timestamp > implicit_item_time_dict[item_id][0]:
                    implicit_item_time_dict[item_id][0] = timestamp
                implicit_item_time_dict[item_id][1] += 1
            else:
                implicit_item_time_dict[item_id] = [timestamp, 1]
                # khoi tao dict gom key => mang 2 phan tu [timestamp, stt]
                # => cuoi cung thu duoc dict: gom key (item_id): value: [lan tuong tac cuoi cung ,so luong/thu tu lan tuong tac do]

        if uid in explicit_rating_dict:  # xet trong phan explicit_rating_dict
            for his in explicit_rating_dict[uid]:  # mang cac tuong tac explicit : item_id , time stamp.
                item_id = his[0]
                timestamp = his[1]
                if item_id in explicit_item_time_dict:
                    if timestamp > explicit_item_time_dict[item_id][0]:
                        explicit_item_time_dict[item_id][0] = timestamp
                    explicit_item_time_dict[item_id][1] += 1
                else:
                    explicit_item_time_dict[item_id] = [timestamp, 1]
                # khoi tao dict gom key => mang 2 phan tu [timestamp, stt]
                # => cuoi cung thu duoc dict: gom key (item_id): value: [lan tuong tac cuoi cung ,thu tu lan tuong tac do]

        # write to file
        for i in implicit_item_time_dict:  # moi i trong implicit dict
            if i in explicit_item_time_dict:  # neu no co tuong tac trong ca explicit dict.
                if implicit_item_time_dict[i][0] > explicit_item_time_dict[i][0]:
                    # max timestamp cua item i tren implicit > explicit, gan nguoc time implicit = explicit.
                    implicit_item_time_dict[i][0] = explicit_item_time_dict[i][0]
                csv_writer.writerow([uid, i] + implicit_item_time_dict[i] + explicit_item_time_dict[i])
                # viet ra file ratings.txt:
                # raw_uid,raw_item_id,last_implicit_timestamp,num_implicit,last_explicit_timestamp,num_explicit
            else:  # neu i co implicit ma khong co explicit doi voi user uid
                csv_writer.writerow([uid, i] + implicit_item_time_dict[i] + [-1, 0])
        # doi voi moi j trong explicit
        for j in explicit_item_time_dict:  # neu co explicit ma khong co tuong tac implicit
            if j not in implicit_item_time_dict:  # j chua co trong implicit data
                csv_writer.writerow([uid, j] + explicit_item_time_dict[j] + explicit_item_time_dict[j])
                # raw_uid,raw_item_id,last_explicit_timestamp,num_explicit1,last_explicit_timestamp,num_explicit
    out_file.close()

In [ ]:
cleaned_implicit_data_path = retail_rocket_root_path + '_implicit.clean.txt'
cleaned_explicit_data_path = retail_rocket_root_path + '_explicit.clean.txt'
user_index_dict = retail_rocket_root_path + 'u2index.txt'
item_index_dict = retail_rocket_root_path + 'i2index.txt'
combined_data = retail_rocket_root_path + 'ratings.txt'
gen_ratings_data_with_explicit(cleaned_implicit_data_path, cleaned_explicit_data_path, 
                               user_index_dict, item_index_dict, combined_data)


In [4]:
import csv
from src import settings

retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'
output = retail_rocket_root_path + 'z_temp_ratings.txt'
writefile = open(output, "w")
csv_writer = csv.writer(writefile, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)

In [15]:
import csv
from src import settings
retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'
file1 = open(retail_rocket_root_path + "ratings_new_ids.txt", "r")
number_explicit = 0
number_line = 0
for line in file1.readlines():
    number_line += 1
    line = line.strip()
    elements = line.split("|")
    interacts = elements[2].strip()[1:-1]
    for interact in interacts.split(","):
        val = interact.strip()
        if int(val) == 1:
            number_explicit +=1
    print ("number line: ", number_line, "number explicit: ", number_explicit)
    


('number line: ', 1, 'number explicit: ', 0)
('number line: ', 2, 'number explicit: ', 0)
('number line: ', 3, 'number explicit: ', 0)
('number line: ', 4, 'number explicit: ', 53)
('number line: ', 5, 'number explicit: ', 59)
('number line: ', 6, 'number explicit: ', 59)
('number line: ', 7, 'number explicit: ', 60)
('number line: ', 8, 'number explicit: ', 62)
('number line: ', 9, 'number explicit: ', 62)
('number line: ', 10, 'number explicit: ', 64)
('number line: ', 11, 'number explicit: ', 64)
('number line: ', 12, 'number explicit: ', 64)
('number line: ', 13, 'number explicit: ', 64)
('number line: ', 14, 'number explicit: ', 64)
('number line: ', 15, 'number explicit: ', 64)
('number line: ', 16, 'number explicit: ', 64)
('number line: ', 17, 'number explicit: ', 64)
('number line: ', 18, 'number explicit: ', 64)
('number line: ', 19, 'number explicit: ', 65)
('number line: ', 20, 'number explicit: ', 68)
('number line: ', 21, 'number explicit: ', 68)
('number line: ', 22, 'nu

('number line: ', 1459, 'number explicit: ', 1263)
('number line: ', 1460, 'number explicit: ', 1264)
('number line: ', 1461, 'number explicit: ', 1264)
('number line: ', 1462, 'number explicit: ', 1264)
('number line: ', 1463, 'number explicit: ', 1264)
('number line: ', 1464, 'number explicit: ', 1267)
('number line: ', 1465, 'number explicit: ', 1267)
('number line: ', 1466, 'number explicit: ', 1267)
('number line: ', 1467, 'number explicit: ', 1267)
('number line: ', 1468, 'number explicit: ', 1267)
('number line: ', 1469, 'number explicit: ', 1267)
('number line: ', 1470, 'number explicit: ', 1268)
('number line: ', 1471, 'number explicit: ', 1269)
('number line: ', 1472, 'number explicit: ', 1269)
('number line: ', 1473, 'number explicit: ', 1269)
('number line: ', 1474, 'number explicit: ', 1269)
('number line: ', 1475, 'number explicit: ', 1269)
('number line: ', 1476, 'number explicit: ', 1269)
('number line: ', 1477, 'number explicit: ', 1269)
('number line: ', 1478, 'number

('number line: ', 2657, 'number explicit: ', 2806)
('number line: ', 2658, 'number explicit: ', 2806)
('number line: ', 2659, 'number explicit: ', 2807)
('number line: ', 2660, 'number explicit: ', 2807)
('number line: ', 2661, 'number explicit: ', 2807)
('number line: ', 2662, 'number explicit: ', 2807)
('number line: ', 2663, 'number explicit: ', 2807)
('number line: ', 2664, 'number explicit: ', 2807)
('number line: ', 2665, 'number explicit: ', 2807)
('number line: ', 2666, 'number explicit: ', 2807)
('number line: ', 2667, 'number explicit: ', 2807)
('number line: ', 2668, 'number explicit: ', 2807)
('number line: ', 2669, 'number explicit: ', 2810)
('number line: ', 2670, 'number explicit: ', 2810)
('number line: ', 2671, 'number explicit: ', 2810)
('number line: ', 2672, 'number explicit: ', 2811)
('number line: ', 2673, 'number explicit: ', 2811)
('number line: ', 2674, 'number explicit: ', 2812)
('number line: ', 2675, 'number explicit: ', 2814)
('number line: ', 2676, 'number

('number line: ', 4513, 'number explicit: ', 4886)
('number line: ', 4514, 'number explicit: ', 4886)
('number line: ', 4515, 'number explicit: ', 4886)
('number line: ', 4516, 'number explicit: ', 4886)
('number line: ', 4517, 'number explicit: ', 4886)
('number line: ', 4518, 'number explicit: ', 4886)
('number line: ', 4519, 'number explicit: ', 4886)
('number line: ', 4520, 'number explicit: ', 4886)
('number line: ', 4521, 'number explicit: ', 4887)
('number line: ', 4522, 'number explicit: ', 4887)
('number line: ', 4523, 'number explicit: ', 4887)
('number line: ', 4524, 'number explicit: ', 4887)
('number line: ', 4525, 'number explicit: ', 4887)
('number line: ', 4526, 'number explicit: ', 4887)
('number line: ', 4527, 'number explicit: ', 4887)
('number line: ', 4528, 'number explicit: ', 4887)
('number line: ', 4529, 'number explicit: ', 4887)
('number line: ', 4530, 'number explicit: ', 4887)
('number line: ', 4531, 'number explicit: ', 4887)
('number line: ', 4532, 'number

('number line: ', 6376, 'number explicit: ', 6482)
('number line: ', 6377, 'number explicit: ', 6482)
('number line: ', 6378, 'number explicit: ', 6482)
('number line: ', 6379, 'number explicit: ', 6482)
('number line: ', 6380, 'number explicit: ', 6482)
('number line: ', 6381, 'number explicit: ', 6482)
('number line: ', 6382, 'number explicit: ', 6482)
('number line: ', 6383, 'number explicit: ', 6482)
('number line: ', 6384, 'number explicit: ', 6482)
('number line: ', 6385, 'number explicit: ', 6482)
('number line: ', 6386, 'number explicit: ', 6482)
('number line: ', 6387, 'number explicit: ', 6482)
('number line: ', 6388, 'number explicit: ', 6483)
('number line: ', 6389, 'number explicit: ', 6483)
('number line: ', 6390, 'number explicit: ', 6483)
('number line: ', 6391, 'number explicit: ', 6483)
('number line: ', 6392, 'number explicit: ', 6483)
('number line: ', 6393, 'number explicit: ', 6483)
('number line: ', 6394, 'number explicit: ', 6483)
('number line: ', 6395, 'number

...

('number line: ', 27453, 'number explicit: ', 23742)
('number line: ', 27454, 'number explicit: ', 23742)
('number line: ', 27455, 'number explicit: ', 23742)
('number line: ', 27456, 'number explicit: ', 23742)
('number line: ', 27457, 'number explicit: ', 23742)
('number line: ', 27458, 'number explicit: ', 23743)
('number line: ', 27459, 'number explicit: ', 23743)
('number line: ', 27460, 'number explicit: ', 23744)
('number line: ', 27461, 'number explicit: ', 23744)
('number line: ', 27462, 'number explicit: ', 23744)
('number line: ', 27463, 'number explicit: ', 23744)
('number line: ', 27464, 'number explicit: ', 23744)
('number line: ', 27465, 'number explicit: ', 23746)
('number line: ', 27466, 'number explicit: ', 23747)
('number line: ', 27467, 'number explicit: ', 23750)
('number line: ', 27468, 'number explicit: ', 23755)
('number line: ', 27469, 'number explicit: ', 23755)
('number line: ', 27470, 'number explicit: ', 23756)
('number line: ', 27471, 'number explicit: ', 

('number line: ', 29365, 'number explicit: ', 25455)
('number line: ', 29366, 'number explicit: ', 25455)
('number line: ', 29367, 'number explicit: ', 25455)
('number line: ', 29368, 'number explicit: ', 25463)
('number line: ', 29369, 'number explicit: ', 25463)
('number line: ', 29370, 'number explicit: ', 25463)
('number line: ', 29371, 'number explicit: ', 25463)
('number line: ', 29372, 'number explicit: ', 25463)
('number line: ', 29373, 'number explicit: ', 25467)
('number line: ', 29374, 'number explicit: ', 25467)
('number line: ', 29375, 'number explicit: ', 25467)
('number line: ', 29376, 'number explicit: ', 25467)
('number line: ', 29377, 'number explicit: ', 25467)
('number line: ', 29378, 'number explicit: ', 25467)
('number line: ', 29379, 'number explicit: ', 25467)
('number line: ', 29380, 'number explicit: ', 25467)
('number line: ', 29381, 'number explicit: ', 25467)
('number line: ', 29382, 'number explicit: ', 25467)
('number line: ', 29383, 'number explicit: ', 

('number line: ', 31319, 'number explicit: ', 26829)
('number line: ', 31320, 'number explicit: ', 26829)
('number line: ', 31321, 'number explicit: ', 26829)
('number line: ', 31322, 'number explicit: ', 26829)
('number line: ', 31323, 'number explicit: ', 26829)
('number line: ', 31324, 'number explicit: ', 26829)
('number line: ', 31325, 'number explicit: ', 26829)
('number line: ', 31326, 'number explicit: ', 26829)
('number line: ', 31327, 'number explicit: ', 26829)
('number line: ', 31328, 'number explicit: ', 26829)
('number line: ', 31329, 'number explicit: ', 26829)
('number line: ', 31330, 'number explicit: ', 26829)
('number line: ', 31331, 'number explicit: ', 26829)
('number line: ', 31332, 'number explicit: ', 26829)
('number line: ', 31333, 'number explicit: ', 26829)
('number line: ', 31334, 'number explicit: ', 26829)
('number line: ', 31335, 'number explicit: ', 26829)
('number line: ', 31336, 'number explicit: ', 26829)
('number line: ', 31337, 'number explicit: ', 

('number line: ', 33254, 'number explicit: ', 28190)
('number line: ', 33255, 'number explicit: ', 28190)
('number line: ', 33256, 'number explicit: ', 28190)
('number line: ', 33257, 'number explicit: ', 28190)
('number line: ', 33258, 'number explicit: ', 28190)
('number line: ', 33259, 'number explicit: ', 28190)
('number line: ', 33260, 'number explicit: ', 28190)
('number line: ', 33261, 'number explicit: ', 28190)
('number line: ', 33262, 'number explicit: ', 28192)
('number line: ', 33263, 'number explicit: ', 28193)
('number line: ', 33264, 'number explicit: ', 28193)
('number line: ', 33265, 'number explicit: ', 28193)
('number line: ', 33266, 'number explicit: ', 28193)
('number line: ', 33267, 'number explicit: ', 28193)
('number line: ', 33268, 'number explicit: ', 28193)
('number line: ', 33269, 'number explicit: ', 28193)
('number line: ', 33270, 'number explicit: ', 28193)
('number line: ', 33271, 'number explicit: ', 28193)
('number line: ', 33272, 'number explicit: ', 

('number line: ', 35004, 'number explicit: ', 29657)
('number line: ', 35005, 'number explicit: ', 29657)
('number line: ', 35006, 'number explicit: ', 29659)
('number line: ', 35007, 'number explicit: ', 29659)
('number line: ', 35008, 'number explicit: ', 29661)
('number line: ', 35009, 'number explicit: ', 29661)
('number line: ', 35010, 'number explicit: ', 29661)
('number line: ', 35011, 'number explicit: ', 29661)
('number line: ', 35012, 'number explicit: ', 29662)
('number line: ', 35013, 'number explicit: ', 29662)
('number line: ', 35014, 'number explicit: ', 29662)
('number line: ', 35015, 'number explicit: ', 29662)
('number line: ', 35016, 'number explicit: ', 29662)
('number line: ', 35017, 'number explicit: ', 29662)
('number line: ', 35018, 'number explicit: ', 29662)
('number line: ', 35019, 'number explicit: ', 29663)
('number line: ', 35020, 'number explicit: ', 29663)
('number line: ', 35021, 'number explicit: ', 29723)
('number line: ', 35022, 'number explicit: ', 

In [ ]:
### Using for splitting test for three datasets.

In [36]:
import csv
from src import settings
retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/recobell/'
file1 = open(retail_rocket_root_path + "ratings_new_ids.txt", "r")

test_file = retail_rocket_root_path + "ratings_test.txt"

number_explicit = 0
test ={}
for line in file1.readlines():
    number_line += 1
    line = line.strip()
    elements = line.split("|")
    item_ids = elements[1].strip()[1:-1]
    itemids =[]
    interactids= []
    
    for item_id in item_ids.split(","):
        itemids.append(int(item_id.strip()))
        
    interacts = elements[2].strip()[1:-1]
    for interact in interacts.split(","):
        val = interact.strip()
        interactids.append(int(val))
    
    if interactids[-1] == 1:
        test[int(elements[0].strip())] =itemids[-1]
    
print "Dict:", test
print "Last: ",len(test.keys())

print("Init writing")
writefile = open(test_file, "w")
csv_writer = csv.writer(writefile, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
for uid in test:
    csv_writer.writerow([uid, test[uid]])

Dict: {32768: 1338, 131073: 479, 196613: 111, 196614: 2130, 7: 198, 128892: 14088, 65545: 342, 32778: 353, 65548: 641, 32781: 479, 48010: 3764, 32784: 6287, 72015: 2060, 65557: 4615, 98327: 586, 65560: 6140, 131098: 1386, 98331: 80330, 131100: 3685, 32799: 1168, 76380: 69644, 65569: 393, 32804: 1126, 163877: 1168, 131114: 3385, 131116: 941, 163886: 305, 196656: 9620, 196658: 4984, 32819: 67, 52: 421, 38238: 10980, 98358: 877, 131081: 5905, 163897: 752, 65594: 48, 196667: 1664, 65596: 3234, 32830: 1168, 196671: 5601, 131137: 12046, 65605: 123, 98374: 2047, 54625: 1768, 72: 184, 12: 2572, 32845: 7067, 163918: 1473, 79: 251, 163920: 22, 131156: 393, 196693: 8887, 163929: 8225, 114703: 1676, 163933: 1928, 98398: 3845, 131088: 1104, 98404: 14678, 5478: 869, 32870: 23766, 65639: 393, 196712: 481, 131177: 1444, 65642: 1713, 107: 3381, 109: 357, 65651: 448, 98422: 242, 98423: 405, 196728: 4835, 196730: 2970, 65659: 124, 65660: 393, 98431: 2747, 196737: 8, 163970: 675, 131: 1046, 98436: 31, 273

In [43]:
# writing into train file.
import csv
from src import settings
retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'
file1 = open(retail_rocket_root_path + "ratings_new_ids.txt", "r")

train_file = retail_rocket_root_path + "ratings_train.txt"

number_explicit = 0
train_items ={}
train_interacts ={}
for line in file1.readlines():
    number_line += 1
    line = line.strip()
    elements = line.split("|")
    
    item_ids = elements[1].strip()[1:-1]
    itemids =[]
    interactids= []
    
    for item_id in item_ids.split(","):
        itemids.append(int(item_id.strip())) # list
        
    interacts = elements[2].strip()[1:-1]
    for interact in interacts.split(","):
        val = interact.strip()
        interactids.append(int(val)) # list

    train_items[int(elements[0].strip())] =itemids[:-1]
    train_interacts[int(elements[0].strip())] = interactids[:-1]
#     print line
#     print train_items
#     print train_interacts
#     break
# print "Dict:", train
print "Last: ",len(train_items.keys())

print("Init writing")
writefile = open(train_file, "w")
csv_writer = csv.writer(writefile, delimiter='|', quotechar='', quoting=csv.QUOTE_NONE)
for uid in train_items:
    csv_writer.writerow([uid, train_items[uid], train_interacts[uid]])

Last:  36751
Init writing
